# Reto 01 - Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este Notebook en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"
Hemos incluido un archivo de entorno de ejemplo para tu referencia. Puedes encontrar el archivo `.env.sample` en la raiz de este repositorio o en la carpeta `Student/Resources` dentro de la documentación del desafío. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como `.env`.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de implementación del modelo que configuraste en Azure OpenAI.

**NOTA:** Ten en cuenta que algunos modelos de AOAI podrían estar obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este primer desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta: [Modelos del Servicio Azure OpenAI Service](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [45]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [46]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.


###### **Temperatura**
La temperatura varía de 0 a 2. Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA. Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.


El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos al mismo tiempo.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Frequency_penalty**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de Azure OpenAI y el [punto final de conexión (endpoint) de chat completions](https://platform.openai.com/docs/guides/chat). 

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.


In [78]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.5, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [55]:
def get_completion_from_messages(messages, model=chat_model, temperature=0.7):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [56]:
messages =  f"""
Cuentame un chiste.
"""
response = get_chat_completion(messages)
print(response)

¿Por qué los pájaros vuelan hacia el sur en invierno?
Porque es


In [58]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'porque la luna tiene huecos'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Porque la comió un ratón de queso. (Translation: Why does the moon have holes? Because it was eaten by a cheese mouse.)


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de finalización de texto (completions) para una temperatura muy alta, entre (0,1).
2. Crea una función de finalización de texto (completions) para un valor muy pequeño de max_tokens a fin de obtener respuestas más cortas.
3. Crea funciones de finalización (completions) con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de finalización (completions) que has creado y compara los resultados obtenidos con las pruebas anteriores.

In [66]:
# Prueba aquí la función de completación para una temperatura muy alta entre (0,1)
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.9, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

messages =  f"""
Cuentame un chiste.
"""
response = get_chat_completion(messages)
print(response)

¿Por qué el tomate se sonrojó? Porque vio la ensalada con el pepino sin ropa.


In [72]:
# Prueba aquí la función de completación para un valor max_tokens muy bajo para obtener respuestas más cortas
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0.4, # this is the degree of randomness of the model's output
        max_tokens = 50,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

messages =  f"""
Cuentame un chiste.
"""
response = get_chat_completion(messages)
print(response)

¿Por qué los pájaros vuelan hacia el sur en invierno?
Porque es demasiado lejos para caminar.


In [ ]:
# Prueba aquí las funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).
def get_completion_from_messages(messages, model=chat_model, frequency_penalty = 0, temperature=0.7):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, 
        frequency_penalty=frequency_penalty
    )
    return response.choices[0].message["content"]

messages =  f"""
Cuentame un chiste.
"""
response = get_chat_completion(messages)
print(response)



¿Por qué los astronautas no pueden tomar soda en el espacio? 

Porque se les sale por las burbujas.


In [76]:
# Prueba aquí las funciones de completación con 2 valores diferentes de penalización por diversidad entre (0,2).
def get_completion_from_messages(messages, model=chat_model, frequency_penalty = 2, temperature=0.7):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, 
        frequency_penalty=frequency_penalty
    )

    return response.choices[0].message["content"]

messages =  f"""
Cuentame un chiste.
"""
response = get_chat_completion(messages)
print(response)

¿Por qué los pájaros vuelan hacia el sur en invierno?
Porque es demasiado lejos para caminar.


### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de Tono

In [79]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Sure, here's a joke for you:

Why did the scarecrow win an award?

Because he was outstanding in his field!


In [80]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Pray tell, what do you call a fake noodle? An impasta!


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.


In [83]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.
messages =  [  
    {'role':'system', 'content':'tu eres una asistente que habla como un juglar del siglo xv'},    
    {'role':'assistant', 'content':'Hola, como puedo ayudarte?'},
    {'role':'user', 'content':'cuentame un chiste'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)


¡Pues claro, aquí va uno de los clásicos del siglo XV! 

Un trovador llega a una taberna y comienza a contar sus hazañas amorosas. Un parroquiano lo observa con desdén y le dice: "¿Qué sabes tú del amor? Si ni siquiera conoces el dueño de tu propia espada". El trovador, sin perder la compostura, responde: "Eso no es cierto mi buen señor. Conozco al dueño de mi espada muy bien... es un herrero llamado Pedro que me la vendió hace un par de semanas".


#### 1.2.2 Recordar el Nombre de la Empresa

In [13]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as a chatbot, I don't have access to your personal information. May I suggest checking your employment contract or contacting your HR department for assistance with this?


#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [84]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hola, mi nombre es Ana y trabajo en codigo facilito'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Yes, of course! You work at Código Facilito. It's a company that provides online courses to people who want to learn about computer programming and other technology-related fields. Is there anything else I can help you with?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**



### 2.1 Escribe instrucciones claras y específicas

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [15]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [ ]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.
import openai

# Define la función para obtener el resumen
def summarize_text(text, model=chat_model):
    prompt = f"""
    Eres un modelo de lenguaje avanzado. Aquí tienes un texto entre delimitadores triple comilla. Por favor, proporciona un resumen conciso de este texto.
    Texto:
    \"\"\"{text}\"\"\"
    Resumen:
    """
    response = openai.ChatCompletion.create(
        engine=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=100,
        top_p=1.0
    )
    return response.choices[0].message["content"]

# Texto original
text = """
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

# Obtener el resumen
summary = summarize_text(text)
print(summary)


Los Juegos Olímpicos de Verano de 2020, también conocidos como Tokio 2020, fueron un evento deportivo internacional celebrado en Tokio, Japón, del 23 de julio al 8 de agosto de 2021, después de ser pospuestos debido a la pandemia de COVID-19. Fue el primer y único evento olímpico en ser realizado sin espectadores oficiales debido a la pandemia. Los Estados Unidos encabezaron la tabla de medallas, seguidos de China y Japón. Se introdujeron nuevos eventos en deportes existentes y se agregaron nuevos deportes al programa olímpico. Fue el primer Juego Olímpico en un año impar y el primer Juego Olímpico de verano en un año no bisiesto desde 1900.


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto a resumir

In [86]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [ ]:
# Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

# Define la función para obtener el resumen
def summarize_text(text, model=chat_model):
    prompt = f"""
    Eres un modelo de lenguaje avanzado. Aquí tienes un texto entre delimitadores triple comilla. Por favor, proporciona un resumen conciso de este texto.
    Texto:
    \"\"\"{text}\"\"\"
    Resumen:
    """
    response = openai.ChatCompletion.create(
        engine=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=100,
        top_p=1.0
    )
    return response.choices[0].message["content"]


# Obtener el resumen
summary = summarize_text(game_review)
print(summary)


Los Juegos Olímpicos de Verano de 2020, también conocidos como Tokio 2020, se llevaron a cabo del 23 de julio al 8 de agosto de 2021 en Tokio, Japón, después de ser pospuestos debido a la pandemia de COVID-19. Fue la primera vez en la historia de los Juegos Olímpicos que se llevaron a cabo sin espectadores oficiales. Los EE


#### Tarea para Estudiantes:
Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

In [90]:
# Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")


# Define la función para obtener el resumen
def summarize_text(text, model=chat_model):
    prompt = f"""
    Texto:
    \"\"\"{text}\"\"\"
    Por favor, extrae del texto los siguientes puntos clave:
    1. Nombre oficial del evento.
    2. Fechas en las que se llevó a cabo.
    3. Lugar donde se realizó.

    """
    response = openai.ChatCompletion.create(
        engine=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=200,
        
    )
    return response.choices[0].message["content"]


# Obtener el resumen
summary = summarize_text(game_review)
print(summary)





1. Nombre oficial del evento: Los Juegos Olímpicos de Verano 2020, oficialmente conocidos como los Juegos de la XXXII Olimpiada y también como Tokyo 2020.
    2. Fechas en las que se llevó a cabo: Desde el 23 de julio hasta el 8 de agosto de 2021, con algunos eventos preliminares que comenzaron el 21 de julio de 2021.
    3. Lugar donde se realizó: Tokyo, Japón.


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [92]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [93]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON


# Define la función para obtener el resumen
def analizar_sentimientos(text, model=chat_model):
    prompt = f"""
    Texto:
    \"\"\"{text}\"\"\"
    Identifica los tipos de emociones y el sentimiento (positivo/negativo) que existen en el texto. 
    Usa un objeto JSON

    """
    response = openai.ChatCompletion.create(
        engine=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=200,
        
    )
    return response.choices[0].message["content"]


# Obtener el resumen
sentimientos = analizar_sentimientos(review)
print(sentimientos)





{
  "emociones": [
    {
      "tipo": "dignidad",
      "sentimiento": "positivo"
    },
    {
      "tipo": "gratitud",
      "sentimiento": "positivo"
    },
    {
      "tipo": "contrastes",
      "sentimiento": "neutro"
    },
    {
      "tipo": "celebración",
      "sentimiento": "positivo"
    },
    {
      "tipo": "preocupación",
      "sentimiento": "negativo"
    },
    {
      "tipo": "extrañeza",
      "sentimiento": "neutro"
    },
    {
      "tipo": "inclusión",
      "sentimiento": "positivo"
    },
    {
      "tipo": "diversidad",
      "sentimiento": "positivo"
    },
    {
      "tipo": "justicia",
      "sentimiento


#### Desafío para Estudiantes:
Deduce 3 temas (tópicos) de la historia a continuación.

In [96]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [101]:
# Deduce 3 temas (tópicos) de la historia a continuación.

# Define la función para procesar un prompt genérico
def procesar_prompt(text, prompt, model=chat_model):
    full_prompt = f"""
    Texto:
    \"\"\"{text}\"\"\" 
    {prompt}
    """
    response = openai.ChatCompletion.create(
        engine=model,
        messages=[{"role": "user", "content": full_prompt}],
        temperature=0.5,
        
    )
    return response.choices[0].message["content"]


# Identificar tres tópicos
text = story
prompt_topicos = "Identifica los tres tópicos principales mencionados en el texto."
resultado_topicos = procesar_prompt(text, prompt_topicos)
print("Tópicos:", resultado_topicos)


Tópicos: 1. La realización de los Juegos Olímpicos de Tokio 2020 en el año 2021 debido a la pandemia de COVID-19 y las medidas de seguridad implementadas durante el evento.
2. La inclusión de nuevos eventos y deportes en el programa olímpico, así como la destacada actuación de algunos países en la competencia.
3. El alto costo de la organización de los Juegos Olímpicos y el hecho de que Tokio se convirtió en la primera ciudad asiática en albergar los Juegos Olímpicos en dos ocasiones.


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [102]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [103]:
# Escribe aquí el código para la tarea .

text = news
prompt_topicos = "traduce por favor al idioma coreano y al ingles."
traducciones_news = procesar_prompt(text, prompt_topicos)
print("Traducciones:", traducciones_news)

Traducciones: Coreano:
"팔레스타인은 2021년 6월 28일 FINA 포인트 시스템을 기반으로 각각의 개인 종목에서 상위 랭킹 스위머(성별 각각 1명)을 올림픽에 보낼 수 있는 보편 초청을 받았습니다. 예선과 결승 두 라운드로 구성된 대회이며, 예선에서 상위 8개의 리 lay 팀이 결승에 진출합니다. 필요한 경우 수영 경기를 사용하여 무승부를 깨고 다음 라운드로 진출합니다."

Inglés:
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021. The competition consists of two rounds: preliminaries and finals. The top eight relay teams from the preliminaries advance to the finals. If necessary, swimming races are used to break ties and advance to the next round."


#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [104]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [106]:
# Escribe aquí el código para la tarea .
text = original_message
prompt_topicos = "transforma el texto en una carta de negocios"
carta = procesar_prompt(text, prompt_topicos)
print(carta)

Estimado David,

Es John, espero que te encuentres bien. Quería comentarte acerca de los Juegos Olímpicos, los cuales están generando una gran cantidad de atención y emoción en todo el mundo.

Considero que este evento deportivo podría ser una gran oportunidad para nuestro negocio, ya que podríamos aprovechar la atención mediática para promocionar nuestros productos o servicios. ¿Has considerado alguna estrategia de marketing en relación a los Juegos Olímpicos?

Sería interesante discutir más sobre este tema y cómo podríamos aprovechar esta oportunidad para hacer crecer nuestro negocio. ¿Te parece si nos reunimos para hablar al respecto?

Espero tu respuesta.

Saludos cordiales,

John


#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [28]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [107]:
# Escribe aquí el código para la tarea .

text = data_json
prompt_topicos = "Convierte el texto en formato html"
formato_html = procesar_prompt(text, prompt_topicos)
print(formato_html)


<pre>
    &lt;html&gt;
    &lt;head&gt;
    &lt;title&gt;The 2020 Summer Olympics Opening Ceremony audience name list&lt;/title&gt;
    &lt;/head&gt;
    &lt;body&gt;
    &lt;ul&gt;
    &lt;li&gt;Name: Shyam &lt;br&gt; Email: shyamjaiswal@gmail.com&lt;/li&gt;
    &lt;li&gt;Name: Bob &lt;br&gt; Email: bob32@gmail.com&lt;/li&gt;
    &lt;li&gt;Name: Jai &lt;br&gt; Email: jai87@gmail.com&lt;/li&gt;
    &lt;/ul&gt;
    &lt;/body&gt;
    &lt;/html&gt;
</pre>


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [108]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [111]:
# Escribe aquí el código para la tarea .

text = check_text
prompt_topicos = "revisa y corrige el texto, manten el idioma en ingles"
corregido = procesar_prompt(text, prompt_topicos)
print(corregido)

"Alongside the main emblem blue, the five other colors used in the branding of the 2020 Games are: Kurenai red, Ai blue, Sakura pink, Fuji purple, and Matsuba green. The competition has three rounds: heats, semifinals, and a final."


#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [112]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [113]:
# Escribe aquí el código para la tarea .

text = review
prompt_topicos = "El cliente tuvo un problema al comprar un boleto para los juegos olimpicos, ten encuenta el sentiment y personaliza la respuesta automatica a este email"
respuesta = procesar_prompt(text, prompt_topicos)
print(respuesta)

Estimado/a cliente,

Lamentamos mucho que esté experimentando problemas con su compra de boletos para los Juegos Olímpicos. Entendemos lo importante que es para usted asegurarse de tener sus boletos a tiempo para disfrutar de los eventos.

Nos gustaría asegurarle que estamos trabajando diligentemente para resolver cualquier problema que pueda haber surgido con su transacción. Por favor, tenga en cuenta que estamos haciendo todo lo posible para que su boleto llegue a usted lo antes posible.

Agradecemos su paciencia y comprensión mientras trabajamos para resolver este problema. Si tiene alguna otra inquietud o pregunta, no dude en ponerse en contacto con nosotros.

Atentamente,

El equipo de atención al cliente


#### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea para Estudiantes:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [114]:
# Escribe aquí el código para la tarea .

text = review
prompt_topicos = "Eres un agente conversacional, y para las conversaciones usaras el texto, iniciaras hola, soy experto en el evento de juegos olimpicos en que te puedo ayudar, si no conoces la respuesta, di lo siento, no tengo la respuesta y responderas cualquier pregunta del user"
conversacion= procesar_prompt(text, prompt_topicos)
print(conversacion)


Hola, soy experto en el evento de Juegos Olímpicos, ¿en qué puedo ayudarte?

User: Compré un boleto para el evento de natación de 100 metros estilo libre para hombres la semana pasada, pero aún no he recibido el boleto. ¿Qué puedo hacer?

Como agente conversacional, le sugiero que primero verifique su correo electrónico o cualquier otra forma de comunicación que haya proporcionado para recibir el boleto. Si no lo encuentra allí, le recomiendo que se comunique con el sitio web o la plataforma donde compró el boleto para obtener más información sobre el estado de su pedido y cómo puede recibir su boleto. ¿Tiene alguna otra pregunta?


### 2.2 Dale tiempo al modelo para "pensar"

#### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [115]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

"<div>Delayed Ticket Delivery for Men's 100 Metre Freestyle Swimming</div>\n\nKeywords: ticket, Men's 100 metre freestyle swimming, delayed delivery, transaction, one week\n\n<table>\n  <tr>\n    <th>Article Publishing Information</th>\n    <th>Keywords</th>\n  </tr>\n  <tr>\n    <td>Issue with Ticket Delivery for Men's 100 Metre Freestyle Swimming</td>\n    <td>ticket, Men's 100 metre freestyle swimming, delayed delivery</td>\n  </tr>\n  <tr>\n    <td>Successful Transaction for Men's 100 Metre Freestyle Swimming Ticket</td>\n    <td>ticket, Men's 100 metre freestyle swimming, transaction</td>\n  </tr>\n  <tr>\n    <td>One Week Delay in Receiving Men's 100 Metre Freestyle Swimming Ticket</td>\n    <td>ticket, Men's 100"

#### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.


2.1 - Prompting de cadena de pensamiento (Chain-of-thought)
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento (Chaining)

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento (Chain-of-Thought Prompting)

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el documento técnico: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [116]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

'Step 1: Identify the Olympic games held at Sydney.\n\nThe Olympic games held at Sydney were the 2000 Summer Olympics.\n\nStep 2: Identify the individual athletes who won multiple medals at the 2000 Summer Olympics.\n\nThere were several individual athletes who won multiple medals at the 2000 Summer Olympics. Some notable ones include:\n\n- Ian Thorpe: The Australian swimmer won five medals (three gold, two silver) at the 2000 Olympics.\n- Lenny Krayzelburg: The American swimmer won three gold medals at the 2000 Olympics.\n- Cathy Freeman: The Australian track and field athlete won two medals (one gold, one silver) at the 2000 Olympics.\n\nStep 3: Determine the most decorated individual athlete at the 2000 Summer Olympics.\n\nTo determine the most decorated individual athlete at the 2000 Summer Olympics, we need to look for an athlete who won more medals than the ones listed above. According to the official Olympic website'

In [117]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

'To find the largest time zone difference between the top two countries who won the most gold medals in the 2020 Tokyo Olympics, we need to determine which two countries won the most gold medals. According to the context, the United States won the most gold medals with 39, and China won the second most with 38. \n\nNext, we need to determine the time zone difference between these two countries. The United States has multiple time zones, but we can use the Eastern Time Zone as a reference since it is the most populous and the location of the capital, Washington D.C. Eastern Time is 13 hours behind Japan Standard Time, which is the time zone used in Tokyo. \n\nChina has only one time zone, China Standard Time, which is 1 hour ahead of Japan Standard Time. \n\nTherefore, the time zone difference between the United States and Japan is 13 hours, and the time zone difference between China and Japan is 1 hour. \n\nThe largest time zone difference between'

#### Táctica 2: Ejemplo de un solo disparo (One-shot)
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [38]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'The United States has 74 medals that are not gold medals (113 total medals - 39 gold medals). \n\nGreat Britain has 42 medals that are not gold medals (64 total medals - 22 gold medals). \n\nTherefore, the United States has 32 more non-gold medals than Great Britain (74 - 42 = 32).'

In [118]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Táctica 3: Razonamiento de cadena de pensamiento del chatbot (chain-of-thought reasoning)
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**Para ingresar la información, aparecerá un cuadro de texto en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [119]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! I am the decision bot. What is the problem or scenario you need help with?



#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [120]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The 2021 Olympics in Tokyo have come to a close, and the medal count has been tallied. As expected, the United States and China dominated the competition, with the US taking home the most medals overall.

The United States won a total of 113 medals, including 39 golds. This was a strong showing for the country, which has consistently been a top performer at the Olympics. China came in second place with 89 medals, including 38 golds. The country has been steadily improving its performance at the Olympics, and this year was no exception.

Japan, as the host country, had a lot to prove and did not disappoint. The country won 27 gold medals and a total of 58 medals overall, making it the third-place winner. Great Britain came in fourth place with 22 gold medals and a total of 64 medals overall. The Russian Olympic Committee (ROC) was fifth with 20 gold medals and 71 medals overall.

While some countries celebrated
"United States and China Dominate Olympics Medal Count"


#### Tarea para Estudiantes (Encadenamiento / Chaining):

Tu tarea ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [121]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'The context does not provide information on which country had the most silver and bronze medals.'

Escribe prompts en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos. Asegúrate de que responda solo con la información que se le proporcionó. Este concepto de fundamentación se presentará con más detalle en el Reto 3.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [125]:
# Escribe aquí el código para la tarea .

prompt = f"""
    Traduce el texto de ingles a español
    Context: {text}
"""

traduccion = get_chat_completion(prompt)


prompt_1 = f"""
    Extrae un JSON con la cantidad de medallas de oro, plata y bronce de cada país mencionados en el contexto.
    Solo proporciona la información explícita del texto.
    Context: {text}
"""
medallas_json = get_chat_completion(prompt_1)

prompt_2 = f"""
    Basándote en el siguiente JSON, identifica el país con la mayor cantidad de medallas de plata y de bronce. 
    Solo utiliza los datos proporcionados en el JSON.

    Datos: {medallas_json}

"""
pais= get_chat_completion(prompt_2)
print(pais)


El país con la mayor cantidad de medallas de plata es Estados Unidos con 41 medallas y el país con la mayor cantidad de medallas de bronce es China con 19 medallas.


## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.